In [108]:
#토양검정정보의 ph 수치(float), 배수등급(str) 추출하기
#군위 21개, 포천 771개의 액셀 파일에서 전부 추출해야함

#1. 파일 불러오는 함수 생성: for문으로 파일들 입력받기
#2. 변수 추출하는 함수 생성: 모든 시트의 동일한 위치에 존재함
#3. 추출한 변수 한 개의 데이터프레임에 합치기
import numpy as np
import pandas as pd


In [109]:
#군위 먼저!!

#1. 파일 불러오기: for문으로 파일들 입력받기
#2. 변수 추출하기: 모든 시트의 동일한 위치에 존재함
#3. 추출한 변수 한 개의 데이터프레임에 합치기
#4. 지저분한 내용 제거해주기
#5. ph값 실수로 바꿔주기
import csv
import re
  
strSupply_water = []
strPh = [] 

for i in range(1, 22):
    path_folder = f'/Users/Star1/Desktop/부트캠프_파이썬 _실습/Python_Practice/토양데이터합치기/파이썬 프로젝트_사과재배적지 분석_csv/토양데이터_군위/gunwi{i}.csv'

    
    with open(path_folder, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter=' ')
        for j, row in enumerate(reader):
            if '\t\t\t\t<th>배수등급</th>' in row:
                next_row_supply = next(reader, None)
                nrs = next_row_supply[0] #re.sub 함수 사용 위해 리스트가 아닌 문자열 추출
                nrs = re.sub(r"""\t\t\t\t<td>""", '', nrs )
                nrs = re.sub(r"</td>", '', nrs)
                
                if not nrs: #빈칸 제거
                    pass
                else:
                    strSupply_water.append(nrs)
            # elif '\t\t\t\t<th>유효토심</th>' in row:
            #     next_row_depth = next(reader, None)
            #     print(next_row_depth)
            elif 'class="s_head">토양분석결과</th>' in row:
                next_row_ph = next(reader, None)
                nrp = next_row_ph[0]
                nrp = re.sub("\t\t\t\t<td>", "", nrp)
                nrp = re.sub("</td>", "", nrp)
                
                if not nrp:
                    pass
                else:
                    nrp = float(nrp)
                    strPh.append(nrp)

print(strSupply_water)
print(strPh)

#결론: strSupply_water랑 strPh 각각에 배수등급과 ph값이 담기게 되었음.


['약간양호', '양호', '매우양호', '양호', '약간불량', '약간양호', '약간양호', '약간양호', '양호', '매우양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간불량', '양호']
[7.7, 6.6, 5.4, 7.6, 6.6, 7.0, 6.5, 6.2, 7.1, 5.4, 6.0, 6.1, 5.6, 6.9, 7.2, 6.4]


In [110]:
import statistics
#1.문자열인 배수등급을 숫자로 바꿔주기
#2.배수등급 최빈값, 중앙값, 전체 평균 도출

dSupply_water = {'매우양호':3, '약간양호':2, '양호':1, '보통':0, '불량':-1 ,'약간불량':-2, '매우불량':-3}
# dictDepth = {'깊음':1, '보통':0, '얕음':-1, '매우얕음':-2}

# for i in range(len(strSupply_water)-1):
#     if strSupply_water[i] in dSupply_water:
#         print(dSupply_water[])

nAvg_water = 0
nSupply_water = []
for elem in strSupply_water:
    if elem in dSupply_water.keys():
        nAvg_water += dSupply_water[elem]
        nSupply_water.append(dSupply_water[elem])

nSupply_water_sort = sorted(nSupply_water)

nMode_water = statistics.mode(nSupply_water_sort) # 배수등급 최빈값

nMedian_water = nSupply_water_sort[len(nSupply_water)//2] #배수등급 중앙값

nAvg_water /= len(strSupply_water) 
nMean_water = round(nAvg_water, 3)#배수등급 평균

print(f"배수등급- 최빈값: {nMode_water}/ 중앙값: {nMedian_water}/ 평균: {nMedian_water}")


#3. ph값 최빈값, 중앙값, 전체 평균 도출
nPh = strPh

nMode_ph = statistics.mode(nPh) # 배수등급 최빈값
nMedian_ph = nPh[len(nPh)//2] #배수등급 중앙값

nMean_ph = 0
for elem in nPh:
    nMean_ph += elem
nMean_ph /= len(nPh)
nMean_ph = round(nMean_ph, 3) #배수등급 평균

print(f"ph값- 최빈값: {nMode_ph}/ 중앙값: {nMedian_ph}/ 평균: {nMean_ph}")


#4. 배수등급과 ph값을 각각의 컬럼으로 바꾸어 하나의 데이터프레임 만들기

df=[]
df_gunwi = pd.DataFrame(df)
df_gunwi['배수등급'] = nSupply_water
df_gunwi['ph농도'] = nPh
print(df_gunwi)

#우연의 일치로 군위는 두 리스트의 인덱스 개수가 같아서 아래와 같은 판다스 작업을 안해줘도 된다.

# nMax_water_ph = max(len(nPh), len(nSupply_water))

# # pd.Series로 데이터프레임의 각 열을 생성
# ph_series = pd.Series(nPh, name='ph농도')
# supply_water_series = pd.Series(nSupply_water, name='배수등급')

# # 데이터프레임 생성
# df_pocheon = pd.DataFrame({'ph농도': ph_series, '배수등급': supply_water_series})
# #df_pocheon = df_pocheon.fillna() #결측치를 바꾸는 함수


배수등급- 최빈값: 2/ 중앙값: 2/ 평균: 2
ph값- 최빈값: 6.6/ 중앙값: 7.1/ 평균: 6.519
    배수등급  ph농도
0      2   7.7
1      1   6.6
2      3   5.4
3      1   7.6
4     -2   6.6
5      2   7.0
6      2   6.5
7      2   6.2
8      1   7.1
9      3   5.4
10     2   6.0
11     2   6.1
12     2   5.6
13     2   6.9
14    -2   7.2
15     1   6.4


In [112]:
#군위 파일 내보내기
df_gunwi.to_csv('./df_gunwi_preproceseed.csv', index=False)


In [113]:
#포천. 군위처럼 그대로 해주기

#군위 먼저!!

#1. 파일 불러오기: for문으로 파일들 입력받기
#2. 변수 추출하기: 모든 시트의 동일한 위치에 존재함
#3. 추출한 변수 한 개의 데이터프레임에 합치기
#4. 지저분한 내용 제거해주기
#5. ph값 실수로 바꿔주기
import csv
import re
  
strSupply_water = []
strPh = [] 

for i in range(2, 772):
    path_folder = f'/Users/Star1/Desktop/부트캠프_파이썬 _실습/Python_Practice/토양데이터합치기/파이썬 프로젝트_사과재배적지 분석_csv/토양데이터_포천/pocheon{i}.csv'

    
    with open(path_folder, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter=' ')
        for j, row in enumerate(reader):
            if '\t\t\t\t<th>배수등급</th>' in row:
                next_row_supply = next(reader, None)
                nrs = next_row_supply[0] #re.sub 함수 사용 위해 리스트가 아닌 문자열 추출
                nrs = re.sub(r"""\t\t\t\t<td>""", '', nrs )
                nrs = re.sub(r"</td>", '', nrs)
                
                if not nrs: #빈칸 제거
                    pass
                else:
                    strSupply_water.append(nrs)
            # elif '\t\t\t\t<th>유효토심</th>' in row:
            #     next_row_depth = next(reader, None)
            #     print(next_row_depth)
            elif 'class="s_head">토양분석결과</th>' in row:
                next_row_ph = next(reader, None)
                nrp = next_row_ph[0]
                nrp = re.sub("\t\t\t\t<td>", "", nrp)
                nrp = re.sub("</td>", "", nrp)
                
                if not nrp:
                    pass
                else:
                    nrp = float(nrp)
                    strPh.append(nrp)

print(strSupply_water)
print(strPh)

#결론: strSupply_water랑 strPh 각각에 배수등급과 ph값이 담기게 되었음.



['약간양호', '약간양호', '약간양호', '양호', '양호', '양호', '양호', '양호', '양호', '약간불량', '매우양호', '매우양호', '양호', '양호', '매우양호', '양호', '매우양호', '양호', '양호', '양호', '양호', '매우양호', '매우불량', '매우불량', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '양호', '양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '양호', '양호', '약간양호', '매우양호', '매우불량', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '양호', '양호', '약간양호', '양호', '약간불량', '약간불량', '약간불량', '약간불량', '양호', '양호', '약간불량', '양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '양호', '양호', '매우양호', '약간양호', '약간양호', '양호', '약간양호', '약간양호', '약간양호', '양호', '양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간양호', '약간불량', '양호', '약간양호', '약간양호', '약간양호', '약간

In [114]:
import statistics
#1.문자열인 배수등급을 숫자로 바꿔주기
#2.배수등급 최빈값, 중앙값, 전체 평균 도출

dSupply_water = {'매우양호':3, '약간양호':2, '양호':1, '보통':0, '불량':-1 ,'약간불량':-2, '매우불량':-3}
# dictDepth = {'깊음':1, '보통':0, '얕음':-1, '매우얕음':-2}

# for i in range(len(strSupply_water)-1):
#     if strSupply_water[i] in dSupply_water:
#         print(dSupply_water[])

nAvg_water = 0
nSupply_water = [] #배수등급 담을 리스트
for elem in strSupply_water:
    if elem in dSupply_water.keys():
        nAvg_water += dSupply_water[elem]
        nSupply_water.append(dSupply_water[elem])

nSupply_water_sort = sorted(nSupply_water)

nMode_water = statistics.mode(nSupply_water_sort) # 배수등급 최빈값

nMedian_water = nSupply_water_sort[len(nSupply_water)//2] #배수등급 중앙값

nAvg_water /= len(strSupply_water) 
nMean_water = round(nAvg_water, 3)#배수등급 평균

print(f"배수등급- 최빈값: {nMode_water}/ 중앙값: {nMedian_water}/ 평균: {nMedian_water}")


#3. ph값 최빈값, 중앙값, 전체 평균 도출
nPh = strPh # ph농도 담을 리스트

nMode_ph = statistics.mode(nPh) # ph 최빈값
nMedian_ph = nPh[len(nPh)//2] #ph 중앙값

nMean_ph = 0
for elem in nPh:
    nMean_ph += elem
nMean_ph /= len(nPh)
nMean_ph = round(nMean_ph, 3) #ph 평균

print(f"ph값- 최빈값: {nMode_ph}/ 중앙값: {nMedian_ph}/ 평균: {nMean_ph}")


#4. 배수등급과 ph값을 각각의 컬럼으로 바꾸어 하나의 데이터프레임 만들기

nMax_water_ph = max(len(nPh), len(nSupply_water))

# pd.Series로 데이터프레임의 각 열을 생성
ph_series = pd.Series(nPh, name='ph농도')
supply_water_series = pd.Series(nSupply_water, name='배수등급')

# 데이터프레임 생성
df_pocheon = pd.DataFrame({'ph농도': ph_series, '배수등급': supply_water_series})
#df_pocheon = df_pocheon.fillna() #결측치를 바꾸는 함수

print(df_pocheon)


# df = pd.DataFrame(index=range(1001), columns=['ph농도', '배수등급'])
# df_pocheon = df.fillna(0) # 모든 데이터를 0으로 초기화

# # nMax_water_ph = max(len(nPh), len(nSupply_water))

# # df=pd.DataFrame({'ph농도': nPh, '배수등급': nSupply_water}, index = range(nMax_water_ph))


# # df_pocheon = pd.DataFrame({'ph농도':nPh, '배수등급': nSupply_water}, index=range(max(len(nPh), len(nSupply_water))))
# df_pocheon['배수등급'] = nSupply_water
# df_pocheon['ph농도'] = nPh


배수등급- 최빈값: 1/ 중앙값: 1/ 평균: 1
ph값- 최빈값: 6.6/ 중앙값: 5.2/ 평균: 6.413
     ph농도  배수등급
0     5.9     2
1     4.8     2
2     5.9     2
3     7.3     1
4     6.8     1
..    ...   ...
591   NaN     1
592   NaN    -2
593   NaN    -2
594   NaN     1
595   NaN     1

[596 rows x 2 columns]


In [115]:
#포천 파일 내보내기
df_pocheon.to_csv('./df_pocheon_preproceseed.csv', index=False)
